# 分散表現

* 分散表現：疎な分布のベクトルから、低次元で実数のベクトル空間に数学的な「埋め込み」を行って作成した、コンピュータで扱いやすい密なベクトル
* LSA → 「同じ文書に出てくる単語の出現回数」の分布を数学的に処理して「意味」を扱う
* SVD、確率モデル、ニューラルネットワークなどを用いる
* 「私は私。」は、「私」「は」「私」「。」という４つのトークンからなり、「私」「は」「。」という３つのタイプから構成されている
* 語彙レベルの分散表現(word2vec、fastText、Gloveなど)は、単語には固有の意味があると考える→類義語には対応可だが、多義語には対応不可

* ディープラーニング→出現レベルの単語の分散表現(単語の出現する文書ごとにその単語のベクトルが異なるため、多義語にも対応可)
* EIMo、BERTなど

# cos類似度

* 単語の意味的な類似性を求める
* 疎なベクトル同士の計算にも適用可能

$ \cos(\overrightarrow{A},\overrightarrow{B}) = \frac{\sum_{n=1}^{i}a_i b_i}{\sqrt{\sum_{n=1}^{i}a_i^2} \sqrt{\sum_{n=1}^{i}b_i^2}} $

* AとBは文書である

In [1]:
# 必要データ
import janome
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

document1='私は秋田犬が大好き。秋田犬は私が大好き。'
document2='私は犬が少し苦手。'
document3='私はぶたとペンギンが好きです。ペンギンは鳥の仲間ですが、水族館で見ることができます。'
document4='隣の家の犬はよく吠える犬です。'
document5='ゴロピカドンが好きです。'
document6='事務室のカギが見つからないと思ったが、よく探したらあった。'
document7='フルーツはどれも大好きですが、私の好物はイチゴです。'
document8='りんごもミカンもパイナップルも、どれもそれぞれとても美味しい果物です。'
document9='私はフルーツが好きで、よくフルーツパーラーに食べに行きます。'
document10='ぶどうとメロンは必需品。'
documents=[document1, document2,document3, document4, document5, document6, document7, document8, document9, document10]

t = Tokenizer()

def make_corpus(documents):
  result_corpus=[]
  for adocument in documents:
    words=[token for token in t.tokenize(adocument, wakati=True)]
    text=" ".join(words)
    result_corpus.append(text)
  return result_corpus

corpus=make_corpus(documents)

vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b', sublinear_tf=True, use_idf=True)
X = vectorizer.fit_transform(corpus)
svd = TruncatedSVD(n_components=7, n_iter=5, random_state=42)
newX=svd.fit_transform(X)

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
A = newX[0]  #「私は秋田犬が大好き。秋田犬は私が大好き。」
B = newX[1]  #「私は犬が少し苦手。」
print(cosine_similarity(A.reshape(1,-1), B.reshape(1,-1)))  #reshapeで1行7列の行列に変換、-1を引数として適切な次元数にする

[[0.98469024]]


In [3]:
C=newX[9]  #「ぶどうとメロンは必需品。」
C

array([ 0.17729441, -0.09072649, -0.20928661,  0.73785634, -0.5761692 ,
       -0.09609122,  0.03714835])

In [4]:
print(cosine_similarity(A.reshape(1,-1), C.reshape(1,-1)))

[[0.07283883]]


In [5]:
#PyTorchによるcos類似度の算出
import torch
import torch.nn.functional as F
A = torch.FloatTensor(newX[0])
B = torch.FloatTensor(newX[1])
F.cosine_similarity(A, B, dim=0)  #dim：何次元目でcos類似度を計算するか、今回は(x,y,z)というベクトルの場合のxで計算
                                  #テンソル：３次元以上の配列

tensor(0.9847)

# word2vec

* word2vec：ニューラルネットワークを使って分散表現を作成する特定の技術と、その技術を使って作られた分散表現のこと
     * 単語の言語コンテキストを再構築するように訓練された浅い2層ニューラルネットワークであり、大きなコーパスを受け取って１つのベクトル空間を生成する
* 単語を密なベクトルとして表現可能
* 加算構成性：ある単語の分散表現を他の単語の分散表現の和で表せる性質　（例 ： 王 － 男 ＋ 女 ＝ 女王）
* CBOW：文脈が与えられたときにその中心にある単語を当てるアルゴリズム
     * コンテキスト単語の順序は問わない
* skip-gram：中心にある単語が与えられたときにその文脈を当てるアルゴリズム
     * 現在の単語に近ければ近いほど、コンテキスト単語の重みを大きくする

* Word2Vecのパラメータ
     * Word2Vec(sentences, sg=1, size=100, window=5, min_count=1)
     * sentences：文書
     * sg：1ならskip-gram,0ならCBOWで学習
     * size：何次元の分散表現を獲得するかを指定
     * window：コンテクストとして認識する前後の単語数を指定
     * min_count：指定の数以下の出現回数の単語は無視する


In [6]:
import os
cwd = os.getcwd()
print(cwd)

c:\Users\snaka\OneDrive\デスクトップ\自然言語処理入門


In [7]:
datafile="C:\\Users\\snaka\\Desktop\\自然言語処理入門\\data.txt"
wakatifile="C:\\Users\\snaka\\Desktop\\自然言語処理入門\\wakati.txt"

with open(wakatifile, 'w', encoding="utf-8") as f2: 

  with open(datafile, 'r', encoding="utf-8") as f1: 
    for line in f1:
      for token in t.tokenize(line, wakati=True):
        f2.write(token+" ")
      f2.write("\n")

In [8]:
from gensim.models import word2vec
from gensim.models import Word2Vec
sentences = word2vec.LineSentence(wakatifile)  #単語の数の配列を作る
model = Word2Vec(sentences)

In [9]:
print(model.wv['犬'])

[ 7.00743496e-03 -6.34719327e-04  7.73871969e-03 -8.90236627e-03
 -8.10148474e-03 -8.30794126e-03 -3.23709147e-03  7.42293196e-03
 -5.43841720e-03 -1.02270199e-02  8.78741872e-03 -5.49487164e-03
  8.67560878e-03 -4.39395197e-03  5.19383606e-03  4.26840642e-03
  3.37211415e-03 -3.85301909e-03  2.67454702e-03 -1.10655986e-02
 -1.93760032e-03 -1.93877437e-03  9.05259885e-03 -4.69297403e-03
 -7.33597996e-03  4.25468851e-03 -2.09917198e-03 -3.13549279e-03
  6.33709691e-03  4.34396090e-03 -2.82557006e-03 -5.05995013e-05
  1.07623767e-02  5.80946682e-03  6.19873824e-03  6.03737356e-03
  2.51220819e-03 -1.71260757e-03 -7.36611849e-03 -1.30660587e-03
 -1.28824543e-03 -7.99243746e-04 -6.76895538e-03  8.01676139e-03
 -5.89640951e-03 -7.95082841e-03 -3.24018695e-03 -2.36023869e-03
 -7.35935429e-03  1.68108940e-03 -4.29904601e-03 -1.98503747e-03
 -7.69211212e-03  1.66085199e-03  3.34484014e-03 -3.46983550e-04
 -4.65169502e-03 -1.96695840e-03  6.78372057e-03  3.70097975e-03
 -9.47262440e-03 -3.62573

In [21]:
from gensim.models import KeyedVectors
model = KeyedVectors.load(cwd + '\\chive-1.2-mc90_gensim\\chive-1.2-mc90.kv') 

In [22]:
model.similarity('葡萄', 'メロン')

0.54135144

In [24]:
model.most_similar('葡萄', topn=5)

[('巨峰', 0.7546089887619019),
 ('果実', 0.6553981304168701),
 ('シャルドネ', 0.6500457525253296),
 ('ピオーネ', 0.6491853594779968),
 ('萄', 0.6364786028862)]

In [25]:
model.most_similar(positive=['日本','パリ'], negative=['東京'], topn=3)

[('フランス', 0.7039362192153931),
 ('ヨーロッパ', 0.6617158055305481),
 ('ロッパ', 0.6234431266784668)]

In [26]:
model.most_similar(positive=['未来','困難'], negative=['夢'], topn=3)

[('不可能', 0.5032184720039368),
 ('状況下', 0.4760775566101074),
 ('容易', 0.4758133590221405)]

# doc2vec

* doc2vec：任意の長さの文章を固定長のベクトルに変換する技術
* 文章や文書の分散表現を獲得する
* word2vec→単語の分散表現
* PV-DM：文章のidと単語を複数個渡し、次に出てくる単語を予測するというタスクを解きながら文章の分散表現を獲得するアルゴリズム
     * word2vecのCBOWに対応
* PV-DBOW：文書が与えられたときに、その中に出てくる単語（語順を考慮しない）を予測するネットワークを学習して、文書ベクトルを獲得するアルゴリズム
     * word2vecのskip-gramに対応

In [ ]:
from gensim.models.doc2vec import TaggedDocument
with open('c:\\Users\\snaka\\OneDrive\\デスクトップ\\自然言語処理入門\\wakati2.txt', encoding="utf-8") as f:
  docs = [TaggedDocument(words=data.split(), tags=[i]) for i, data in enumerate(f)]  #空白ごとに区切る

In [ ]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec(docs)

In [ ]:
model.save('mymodel.model') #モデルの保存
model=Doc2Vec.load('mymodel.model') #読み込み

In [ ]:
model.docvecs[0]  #IDが0のベクトルを得る

C:\Users\snaka\AppData\Local\Temp\ipykernel_16196\2021273564.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs[0]


array([-0.00553779, -0.00615797, -0.01012321,  0.00866437,  0.00346033,
       -0.00031075, -0.0094957 , -0.00424911, -0.01039819,  0.00155471,
        0.00277572,  0.00395738, -0.00417691, -0.00287537, -0.00284936,
       -0.00927095,  0.00265095,  0.00909252, -0.0096799 , -0.00425436,
       -0.00352167,  0.00240364, -0.00504206,  0.00239101,  0.00586166,
       -0.00812463, -0.00920627, -0.01002444,  0.00451112, -0.00946636,
        0.00663882,  0.00689971, -0.00588151, -0.00482819, -0.00115095,
        0.00220524, -0.00160562, -0.00870074, -0.00377032,  0.00171615,
       -0.00159121, -0.00745074,  0.00404443, -0.0088798 ,  0.00295637,
       -0.00482227,  0.00038301, -0.00251194,  0.00563254, -0.00768922,
       -0.00187831, -0.00086241, -0.00675723, -0.00711295, -0.00195158,
        0.008772  , -0.00105768,  0.00341701, -0.00595282,  0.00893064,
        0.00316198,  0.00935392,  0.00514747, -0.00396496,  0.00191563,
       -0.00355881,  0.00564387,  0.00216027, -0.00310016, -0.00

In [ ]:
model.docvecs.most_similar(0, topn=2)  #IDが0の文書に最も類似した文書がIDが30の文書

C:\Users\snaka\AppData\Local\Temp\ipykernel_16196\2309477780.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs.most_similar(0, topn=2)  #IDが0の文書に最も類似した文書がIDが30の文書


[(30, 0.2341257929801941), (40, 0.2320420742034912)]

In [ ]:
newdoc = ['私', 'は', '秋田', '犬', 'が', '大好き', '。']
model.infer_vector(newdoc)  #newdocのベクトルを推論

array([ 2.0941845e-03,  1.6542207e-03, -2.2828518e-03, -2.7250140e-03,
       -2.6255255e-03,  3.7480987e-04, -1.5203730e-03,  1.9013119e-03,
       -9.5949741e-04,  3.4623079e-03,  3.7278903e-03,  3.9796829e-03,
        4.6212226e-03, -5.9217640e-04,  1.8265432e-03, -5.0215744e-03,
        3.6066663e-03, -3.8924334e-03, -1.3782302e-03, -2.2157456e-03,
       -1.9296191e-03, -1.0536737e-03,  1.1909648e-03, -3.8748612e-03,
       -8.0215465e-04, -1.4819851e-03, -5.2664131e-03, -3.5088959e-03,
        2.8387350e-03, -2.0329726e-03, -2.5814935e-03,  3.0649755e-07,
       -3.8580671e-03, -4.6454724e-03, -3.4133866e-03,  2.4967412e-03,
       -3.5047743e-03, -4.9026976e-03, -4.5988583e-03, -2.0452722e-03,
        4.7840454e-04, -3.3606091e-03,  5.6390767e-04,  2.1878502e-03,
       -5.1187846e-04,  2.8443641e-03, -3.5668560e-03,  4.5152384e-04,
       -4.3709381e-03, -4.0892352e-05, -9.1278652e-04,  2.0399103e-03,
       -3.8291735e-03,  1.7619805e-03,  2.5473309e-03,  3.3912859e-03,
      